In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.amp
from torch.utils.checkpoint import checkpoint
import numpy as np
from contextlib import nullcontext
import math

torch.cuda.empty_cache()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# CONFIG – optimized for H100 speed (fast epochs)
triality = 3
dim = 384
latent_dim = 8
seq_len = 1024  # collision "hits" proxy
batch_size = 32
epochs = 20000
lr = 5e-5
use_amp = True
use_checkpoint = True

# Synthetic Higgs boson proxy (4-lepton golden channel + background noise/occlusion)
# Real-like curved helical tracks for signal + random hits for background
events = []
for b in range(batch_size):
    # Signal: 4 leptons (curved paths in magnetic field proxy)
    t = torch.linspace(0, 2*math.pi, seq_len, device=device)
    lepton1 = torch.stack([torch.sin(t), torch.cos(t), t/10], dim=-1) * 50
    lepton2 = torch.stack([torch.sin(t + math.pi/2), torch.cos(t + math.pi/2), t/10], dim=-1) * 40
    lepton3 = torch.stack([torch.sin(t + math.pi), torch.cos(t + math.pi), t/10], dim=-1) * 30
    lepton4 = torch.stack([torch.sin(t + 3*math.pi/2), torch.cos(t + 3*math.pi/2), t/10], dim=-1) * 20

    # Stack leptons + background noise
    signal = torch.cat([lepton1, lepton2, lepton3, lepton4], dim=-1)  # (seq_len, 12 features proxy)
    background = torch.randn_like(signal) * 5  # noise
    event = signal + background

    # Normalize
    event = (event - event.min()) / (event.max() - event.min() + 1e-6)
    events.append(event)

events = torch.stack(events).to(device)  # (batch, seq_len, features)

# Project to dim
proj = nn.Linear(events.shape[-1], dim).to(device)

# E8 roots – precompute
def get_e8_roots():
    roots = []
    for i in range(8):
        for j in range(i+1, 8):
            for signs in [(1,1), (1,-1), (-1,1), (-1,-1)]:
                v = torch.zeros(8)
                v[i] = signs[0]; v[j] = signs[1]
                roots.append(v); roots.append(-v)
    for signs in range(1 << 8):
        v = torch.tensor([(1 if (signs & (1<<k)) else -1) for k in range(8)], dtype=torch.float32) * 0.5
        if bin(signs).count('1') % 2 == 0:
            roots.append(v); roots.append(-v)
    roots = torch.stack(roots[:240])
    return roots / roots.norm(dim=-1, keepdim=True)

e8_roots = get_e8_roots().to(device)

# Triality Cycle Block (detached pump scalar)
class HiggsCycleBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.proj = nn.Linear(latent_dim, dim // triality, bias=False)
        self.register_buffer('roots', e8_roots)

    def forward(self, x, step):
        pos_emb = self.roots[torch.arange(x.shape[1], device=device) % 240]
        low_dim = self.proj(pos_emb)
        emb = low_dim.repeat(1, triality)
        with torch.no_grad():
            pump_scalar = 0.8 * math.sin(step * 0.006 * 2 * math.pi)
        pump = torch.full((1, x.shape[1], 1), pump_scalar, device=device)
        emb_broadcast = emb.unsqueeze(0)
        x_rot1 = x * (emb_broadcast.cos() + pump)
        x_rot2 = torch.roll(x_rot1, shifts=1, dims=1) * emb_broadcast.sin()
        x_rot3 = torch.roll(x_rot2, shifts=1, dims=1) * emb_broadcast.cos()
        fused = (x_rot1 + x_rot2 + x_rot3) / triality
        return fused

# Dummy cycle for ablation
class DummyCycle(nn.Module):
    def forward(self, x, step=None):
        return x

# Model with ablation support
class E8HiggsFusion(nn.Module):
    def __init__(self, depth=32, use_triality=True):
        super().__init__()
        self.use_triality = use_triality
        self.cycle = HiggsCycleBlock() if use_triality else DummyCycle()
        self.layers = nn.ModuleList([nn.MultiheadAttention(dim, triality if use_triality else 8, batch_first=True) for _ in range(depth)])
        self.norm = nn.LayerNorm(dim)
        self.head = nn.Linear(dim, dim)

    def forward(self, x, step):
        x = self.cycle(x, step)
        for layer in self.layers:
            if use_checkpoint:
                attn, _ = checkpoint(layer, x, x, x, use_reentrant=False)
            else:
                attn, _ = layer(x, x, x)
            x = x + self.norm(attn)
        return x

# Models
model = E8HiggsFusion(use_triality=True).to(device)
model_ablation = E8HiggsFusion(use_triality=False).to(device)

opt = torch.optim.AdamW(model.parameters(), lr=lr)
sch_model = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=epochs)
sch_model_step_after_opt = True # Flag to ensure scheduler step is called correctly
scaler = torch.amp.GradScaler('cuda') if use_amp else nullcontext()

opt_ablation = torch.optim.AdamW(model_ablation.parameters(), lr=lr)
sch_ablation = torch.optim.lr_scheduler.CosineAnnealingLR(opt_ablation, T_max=epochs)
sch_ablation_step_after_opt = True # Flag to ensure scheduler step is called correctly
scaler_ablation = torch.amp.GradScaler('cuda') if use_amp else nullcontext()

loss_fn = nn.MSELoss()

loss_hist = []
loss_abl_hist = []

for epoch in range(epochs):
    # Re-compute clean_data and real_data each epoch to ensure fresh graph for backprop
    clean_data = proj(events)
    missing_rate = torch.linspace(0.4, 0.7, batch_size, device=device).view(batch_size, 1, 1)
    mask = torch.rand_like(clean_data) < missing_rate
    real_data = clean_data.clone().masked_fill(mask, 0)
    target = clean_data

    opt.zero_grad(set_to_none=True)
    opt_ablation.zero_grad(set_to_none=True)

    with torch.amp.autocast(device_type='cuda', dtype=torch.float16) if use_amp else nullcontext():
        recon = model(real_data, epoch)
        loss = loss_fn(recon, target)

        recon_abl = model_ablation(real_data, epoch)
        loss_abl = loss_fn(recon_abl, target)

    scaler.scale(loss).backward(retain_graph=True) if use_amp else loss.backward(retain_graph=True)
    scaler.unscale_(opt) if use_amp else None
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1e6)
    scaler.step(opt) if use_amp else opt.step()
    if sch_model_step_after_opt: # Only step the scheduler after the optimizer
      sch_model.step()
    scaler.update() if use_amp else None

    scaler_ablation.scale(loss_abl).backward() if use_amp else loss_abl.backward()
    scaler_ablation.unscale_(opt_ablation) if use_amp else None
    torch.nn.utils.clip_grad_norm_(model_ablation.parameters(), 1e6)
    scaler_ablation.step(opt_ablation) if use_amp else opt_ablation.step()
    if sch_ablation_step_after_opt: # Only step the scheduler after the optimizer
      sch_ablation.step()
    scaler_ablation.update() if use_amp else None

    loss_hist.append(loss.item())
    loss_abl_hist.append(loss_abl.item())

    if epoch % 500 == 0:
        print(f"Epoch {epoch} | Triality Loss {loss.item():.6f} | Ablation Loss {loss_abl.item():.6f}")

# Final Sigma Test
triality_mean = np.mean(loss_hist)
abl_mean = np.mean(loss_abl_hist)
std = np.std(loss_hist + loss_abl_hist)
sigma = (abl_mean - triality_mean) / std if std > 0 else 0

print(f"Final Sigma (Triality vs Ablation): {sigma:.2f} (higher = triality advantage)")

print("Sim complete — epochs + sigma test done")

Using device: cuda
Epoch 0 | Triality Loss 33.365021 | Ablation Loss 29.105597
Epoch 500 | Triality Loss 0.027632 | Ablation Loss 0.051425
Epoch 1000 | Triality Loss 0.020081 | Ablation Loss 0.046025
Epoch 1500 | Triality Loss 0.017011 | Ablation Loss 0.042758
Epoch 2000 | Triality Loss 0.015674 | Ablation Loss 0.041308
Epoch 2500 | Triality Loss 0.014785 | Ablation Loss 0.040499
Epoch 3000 | Triality Loss 0.014335 | Ablation Loss 0.039508
Epoch 3500 | Triality Loss 0.013421 | Ablation Loss 0.039827
Epoch 4000 | Triality Loss 0.013730 | Ablation Loss 0.039365
Epoch 4500 | Triality Loss 0.013257 | Ablation Loss 0.038801
Epoch 5000 | Triality Loss 0.013237 | Ablation Loss 0.037974
Epoch 5500 | Triality Loss 0.012789 | Ablation Loss 0.037736
Epoch 6000 | Triality Loss 0.012404 | Ablation Loss 0.036622
Epoch 6500 | Triality Loss 0.011856 | Ablation Loss 0.035824
Epoch 7000 | Triality Loss 0.011261 | Ablation Loss 0.034985
Epoch 7500 | Triality Loss 0.010485 | Ablation Loss 0.034269
Epoch 8